In [1]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.autograd import Variable
import numpy as np
import random
from PIL import Image
from ipywidgets import FloatProgress
from IPython.display import display
from __future__ import print_function

from model import ModelDefinition
from dataset import ReadImages, collection
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

Process the dataset :
We have to compute the number of class, and the mean and std for image normalization

Read the dataset and compute the mean and std dev :

In [2]:
trainset = ReadImages.readImageswithPattern('/video/CLICIDE', lambda x:x.split('/')[-1].split('-')[0])
testset = ReadImages.readImageswithPattern('/video/CLICIDE/test/', lambda x:x.split('/')[-1].split('-')[0])

In [3]:
def MeanAndStd(imageList, fname=None):
    imageListOpen = ReadImages.openAll(trainset, (225,225))
    m = collection.ComputeMean(imageListOpen)
    print("Mean : ", m)
    s = collection.ComputeStdDev(imageListOpen, m)
    print("std dev : ", s)
    if not fname is None:
        with open(fname, "w") as f:
            f.write('%.3f %.3f %.3f' %m)
            f.write('\n')
            f.write('%.3f %.3f %.3f' %(s))
    return m,s

In [4]:
def readMeanStd(fname='data/cli.txt'):
    with open(fname) as f:
        mean = map(float, f.readline().split(' '))
        std = map(float, f.readline().split(' '))
    return mean, std

In [5]:
m, s = readMeanStd()

Define the network as class (from nn.Module) :

Training

In [6]:
#create the network
mymodel = ModelDefinition.Maxnet()
ModelDefinition.copyParameters(mymodel, models.alexnet(pretrained=True))

In [18]:
#define the optimizer to only the classifier with lr of 1e-3

optimizer=optim.SGD([
                {'params': mymodel.classifier.parameters()},
                {'params': mymodel.features.parameters(), 'lr': 0.0}
            ], lr=1e-3, momentum=0.9)

In [7]:
listLabel = [t[1] for t in trainset if not 'wall' in t[1]]

In [8]:
labels = list(set(listLabel))

In [9]:
for i in range(len(trainset)):
    trainset[i] = (Image.open(trainset[i][0]), trainset[i][1])

In [10]:
for i in range(len(testset)):
    testset[i] = (Image.open(testset[i][0]), testset[i][1])

In [ ]:
mymodel = torch.load('model-5.cpkt')
ep = 5

In [17]:
ep = 2

In [19]:
#mymodel.cuda()
#mymodel = best.train()
criterion = nn.loss.CrossEntropyLoss()
#optimizer = optim.SGD(mymodel.parameters(), lr=0.0001, momentum=0.9)

#trainset, imagesList = readImages("CliList.txt")
#testset, imagesTest = readImages("CliListTest.txt")
#labels = open("CliConcept.txt").read().splitlines()

mymodel.train().cuda()

imageTransform = transforms.Compose( (transforms.Scale(300), transforms.RandomCrop(225), transforms.ToTensor(), transforms.Normalize(m,s)) )
testTransform = transforms.Compose( (transforms.Scale(225), transforms.CenterCrop(225), transforms.ToTensor(), transforms.Normalize(m,s)))
batchSize = 64
bestScore = 0
for epoch in range(ep, 50): # loop over the dataset multiple times
    running_loss = 0.0
    random.shuffle(trainset)    
    for i in range(len(trainset)/batchSize):
        
        # get the inputs
        inputs = torch.Tensor(batchSize,3,225,225).cuda()
        for j in range(batchSize):
            inputs[j] = imageTransform(trainset[j+i*batchSize][0])
        inputs = Variable(inputs)
        
        #get the labels
        lab = Variable(torch.LongTensor([labels.index(trainset[j+i*batchSize][1]) for j in range(batchSize)]).cuda())
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = mymodel(inputs)
        loss = criterion(outputs, lab)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.data[0]
        if i % 10 == 9: # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss / 10))
            running_loss = 0.0
            
        if i % 50 == 49: #test every 20 mini-batches
            print('test :')
            mymodel = mymodel.eval()
            correct = 0
            tot = 0
            cpt = 0
            for j in range(len(testset)/batchSize):
                inp = torch.Tensor(batchSize,3,225,225).cuda()
                for k in range(batchSize):
                    inp[k] = testTransform(testset[j*batchSize+k][0])
                    cpt += 1
                outputs = mymodel(Variable(inp))
                _, predicted = torch.max(outputs.data, 1)
                predicted = predicted.tolist()
                for k in range(batchSize):
                    if (testset[j*batchSize+k][1] in labels):
                        correct += (predicted[k][0] == labels.index(testset[j*batchSize+k][1]))
                        tot += 1
                        
            rest = len(testset)%batchSize
            inp = torch.Tensor(rest,3,225,225).cuda()
            for j in range(rest):
                inp[j] = testTransform(testset[len(testset)-rest+j][0])
            outputs = mymodel(Variable(inp))
            _, predicted = torch.max(outputs.data, 1)
            predicted = predicted.tolist()
            for j in range(rest):
                if (testset[len(testset)-rest+j][1] in labels):
                   correct += (predicted[j][0] == labels.index(testset[len(testset)-rest+j][1]))
                   tot += 1
            print("Correct : ", correct, "/", tot)
            if (correct >= bestScore):
                best = mymodel
                bestScore = correct
                torch.save(best, "bestModel.ckpt")
            #else:
            #    mymodel = best
            torch.save(mymodel, "model-"+str(epoch)+".ckpt")
            mymodel = mymodel.train()
            
print('Finished Training')

[3,    10] loss: 0.521
[3,    20] loss: 0.539
[3,    30] loss: 0.418
[3,    40] loss: 0.520
[3,    50] loss: 0.421
test :
Correct :  74 / 165
[4,    10] loss: 0.439
[4,    20] loss: 0.369
[4,    30] loss: 0.361
[4,    40] loss: 0.383
[4,    50] loss: 0.408
test :
Correct :  76 / 165
[5,    10] loss: 0.349
[5,    20] loss: 0.410
[5,    30] loss: 0.329
[5,    40] loss: 0.357
[5,    50] loss: 0.341
test :
Correct :  78 / 165
[6,    10] loss: 0.294
[6,    20] loss: 0.331
[6,    30] loss: 0.348
[6,    40] loss: 0.308
[6,    50] loss: 0.330
test :
Correct :  78 / 165
[7,    10] loss: 0.269
[7,    20] loss: 0.320
[7,    30] loss: 0.312
[7,    40] loss: 0.269
[7,    50] loss: 0.319
test :
Correct :  77 / 165
[8,    10] loss: 0.374
[8,    20] loss: 0.285
[8,    30] loss: 0.274
[8,    40] loss: 0.311
[8,    50] loss: 0.255
test :
Correct :  79 / 165
[9,    10] loss: 0.294
[9,    20] loss: 0.243
[9,    30] loss: 0.242
[9,    40] loss: 0.321
[9,    50] loss: 0.318
test :
Correct :  81 / 165
[10,  

In [ ]:
m = ModelDefinition.SiameseMax()

In [ ]:
im = Image.open(trainset[0][0]).resize( (225, 225))
testTransform = transforms.Compose( (transforms.Scale(225), transforms.ToTensor()))
im = testTransform(im)

In [ ]:
t = torch.Tensor(1,3,225,225)
t[0] = im
output = m( Variable(t), Variable(t) ) 

In [ ]:
optimizer